<a href="https://colab.research.google.com/github/ivkarla/epigame/blob/main/notebooks/anonymize_metadata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
subs_names ={
"ASJ":1,
"BGL":2,
"BRM":3,
"CRD":4,
"CRF":5,
"GTA":6,
"HAF":7,
"HDW":8,
"JQN":9,
"MGM":10,
"MMM":11,
"MRI":12,
"MSF":13,
"PTD":14,
"RGE":15,
"SDA":16,
"SRM":17,
"USA":18,
"VBM":19,
"VCG":20,
"VML":21
}

In [5]:
import pandas as pd

from google.colab import drive
drive.mount('/content/gdrive')
root_folder = "/content/gdrive/"

project_folder = "Shared drives/A&K/data/"
main_folder = root_folder + project_folder

metadata_file = main_folder + "subsmeta.xlsx"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [61]:
# Create DataFrame
df = pd.read_excel(metadata_file)

# Create a dictionary to map each unique channel to an ID

def get_anonymized_channel(contact, contact_id_map, next_id):

    if contact not in contact_id_map:
        contact_id_map[contact] = next_id
        next_id += 1
    return contact_id_map[contact], next_id

# Anonymize the EEG channels
anonymized_channels = []

for channels in df['ch_labels']:

  next_id = 1
  contact_id_map = {}

  anonymized_channel_list = []
  channels_list = channels.strip("[]").split(',')
  channels_list = [channel.strip(" ").strip("'").strip('"') for channel in channels_list]

  for channel in channels_list:

    contact_1, contact_2 = channel.split('-')

    contact_1_id, next_id = get_anonymized_channel(contact_1, contact_id_map, next_id)
    contact_2_id, next_id = get_anonymized_channel(contact_2, contact_id_map, next_id)
    anonymized_channel_list.append(f'{contact_1_id}-{contact_2_id}')

  anonymized_channels.append(anonymized_channel_list)

# Add the anonymized channels to the DataFrame
df['anonymized_channels'] = anonymized_channels


In [62]:
def anonymize_channels(channels):
    contact_mapping = {}
    current_id = 1

    channels_list = channels.split(',')
    channels_list = [channel[2:-1] for channel in channels_list]

    for channel in channels_list:
      # Split the channel into two contacts
      contact_1, contact_2 = channel.split('-')

      if contact_1 not in contact_mapping:
        contact_mapping[contact_1] = current_id
        current_id += 1
      if contact_2 not in contact_mapping:
        contact_mapping[contact_2] = current_id
        current_id += 1

    return contact_mapping

# Apply anonymization for each row in the DataFrame
contact_mapping = {}

for i, row in df.iterrows():
  sub_id = row['sub']
  label_map = anonymize_channels(row['ch_labels'])
  contact_mapping[sub_id] = label_map


In [64]:
def anonymize_resection(resection_list, mapping):
    return [mapping.get(contact) for contact in resection_list]

# Apply the anonymization for the 'ch_resection' column
df['anonymized_ch_resection'] = df.apply(lambda row: anonymize_resection([contact.strip(" ").strip("'").strip('"') for contact in row['ch_resection'].strip("[]").split(',')], contact_mapping[row['sub']]), axis=1)


In [ ]:
df

In [66]:
df.to_excel(main_folder + "anonymized_metadata.xlsx", index=False)


In [71]:
results_concat = pd.read_excel(main_folder + "results-concat.xlsx")

def anonymize_en(en_list, mapping):
  anonymized_en = []
  for channel in en_list:
    contact_1, contact_2 = channel.split('-')
    anonymized_contact_1 = mapping.get(contact_1)
    anonymized_contact_2 = mapping.get(contact_2)

    anonymized_en.append(f"{anonymized_contact_1}-{anonymized_contact_2}")
    return anonymized_en

# Apply the anonymization for the 'EN' column
results_concat['anonymized_EN'] = results_concat.apply(lambda row: anonymize_en([contact.strip(" ").strip("'").strip('"') for contact in row['EN'].strip("[]").split(',')], contact_mapping[row['subject']]), axis=1)

results_concat['anonymized_subject'] = results_concat['subject'].map(subs_names)